# Урок 5. Spark on scala

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

## Задание:

— Используя Spark прочитайте данные из файла csv.

— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

— Отсортируйте данные по общему объему продаж в порядке убывания.

— Выведите результаты на экран.

In [2]:
# Установка необходимых библиотек
!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Импорт необходимых библиотек
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [4]:
# Настройка переменных окружения для Spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [5]:
# Создание SparkSession
spark = SparkSession.builder \
    .appName("Spark Python CSV Example") \
    .master("local[*]") \
    .getOrCreate()

In [6]:
# Определение данных
data = [
    ("1984", "George Orwell", "Science Fiction", 5000, 1949),
    ("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
    ("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
    ("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
    ("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)
]

In [7]:
# Создание DataFrame
columns = ["title", "author", "genre", "sales", "year"]
df = spark.createDataFrame(data, schema=columns)

In [8]:
# Определение пути к файлу
csv_file_path = "/content/books.csv"

In [9]:
# Проверка существования файла и удаление, если существует
if os.path.exists(csv_file_path):
    import shutil
    shutil.rmtree(csv_file_path)

In [10]:
# Сохранение DataFrame в CSV-файл
df.write.option("header", "true").csv(csv_file_path)

In [11]:
# Чтение данных из CSV-файла
books_df = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_file_path)

In [12]:
# Фильтрация данных, чтобы оставить только книги с продажами выше 3000 экземпляров
filtered_df = books_df.filter(col("sales") > 3000)

In [13]:
# Группировка данных по жанру и вычисление общего объема продаж для каждого жанра
grouped_df = books_df.groupBy("genre").agg(sum("sales").alias("total_sales"))

In [15]:
# Сортировка данных по общему объему продаж в порядке убывания
sorted_df = grouped_df.orderBy(col("total_sales").desc())

In [16]:
# Вывод результатов на экран с подписями
print("Original DataFrame:")
books_df.show()

print("Filtered DataFrame (sales > 3000):")
filtered_df.show()

print("Grouped DataFrame by genre with total sales:")
grouped_df.show()

print("Sorted DataFrame by total sales:")
sorted_df.show()

Original DataFrame:
+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
+--------------------+-------------------+---------------+-----+----+

Filtered DataFrame (sales > 3000):
+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fit

In [17]:
# Остановка SparkSession
spark.stop()